## __Classification: Model Training__

In [1]:
## importing necessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

import cv2
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
## xgboost, lightgbm, catboost
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import RandomRotation, RandomContrast, RandomTranslation, RandomBrightness, RandomFlip    
from tensorflow.keras.models import Sequential

In [2]:
## setting up the images directory

images_dir = '../data/classification'
img_dimensions = (112, 112, 3)

print(os.listdir(images_dir))

['0', '1', '2', '3', '4', '5', '6', '7', '8']


In [3]:
## loading dataset with 9 classses stored in 9 independent lists with 9 lists of labels
## the dataset is stored in a folder called "dataset" in the current working directory

def load_dataset(i):
    dataset = []
    labels = []
    
    path = images_dir + '/' + str(i)
    files = os.listdir(path)
    for file in files:
        file_path = os.path.join(path, file)
        img = cv2.imread(file_path)
        if img is None:
            continue
        img = cv2.resize(img, (112, 112))
        dataset.append(img)
        labels.append(i)
    return np.array(dataset), np.array(labels)

In [4]:
class_0, labels_0 = load_dataset(0)
class_1, labels_1 = load_dataset(1)
class_2, labels_2 = load_dataset(2)
class_3, labels_3 = load_dataset(3)
class_4, labels_4 = load_dataset(4)
class_5, labels_5 = load_dataset(5)
class_6, labels_6 = load_dataset(6)
class_7, labels_7 = load_dataset(7)
class_8, labels_8 = load_dataset(8)

In [5]:
data_augmentation = Sequential([
  RandomRotation(0.1),
  RandomContrast(0.1),
  RandomBrightness(0.1),
  RandomTranslation(0.15, 0.15),
  RandomFlip('horizontal')
])

data_augmentation_reduced = Sequential([
  RandomContrast(0.1),
  RandomBrightness(0.1),
  RandomTranslation(0.15, 0.15)
])

In [7]:
def augment(train_images, train_labels):
    augmented_images = []
    augmented_labels = []
    for i in range(train_images.shape[0]):
        augmented_images.append(train_images[i])
        augmented_labels.append(train_labels[i])
        for j in range(4):
            augmented_images.append(data_augmentation(train_images[i]))
            augmented_labels.append(train_labels[i])
    return np.array(augmented_images), np.array(augmented_labels)

def augment_low_low(train_images, train_labels):
    augmented_images = []
    augmented_labels = []
    for i in range(train_images.shape[0]):
        augmented_images.append(train_images[i])
        augmented_labels.append(train_labels[i])
        for j in range(4):
            augmented_images.append(data_augmentation_reduced(train_images[i]))
            augmented_labels.append(train_labels[i])
    return np.array(augmented_images), np.array(augmented_labels)

def augment_more_low(train_images, train_labels):
    augmented_images = []
    augmented_labels = []
    for i in range(train_images.shape[0]):
        augmented_images.append(train_images[i])
        augmented_labels.append(train_labels[i])
        for j in range(7):
            augmented_images.append(data_augmentation_reduced(train_images[i]))
            augmented_labels.append(train_labels[i])
    return np.array(augmented_images), np.array(augmented_labels)

def augment_much_more(train_images, train_labels):
    augmented_images = []
    augmented_labels = []
    for i in range(train_images.shape[0]):
        augmented_images.append(train_images[i])
        augmented_labels.append(train_labels[i])
        for j in range(15):
            augmented_images.append(data_augmentation(train_images[i]))
            augmented_labels.append(train_labels[i])
    return np.array(augmented_images), np.array(augmented_labels)

def normalize(train_images):
    return train_images / 255.0

In [7]:
class_0, labels_0 = augment(class_0, labels_0)
class_1, labels_1 = augment_more_low(class_1, labels_1)
class_2, labels_2 = augment_low_low(class_2, labels_2)
class_3, labels_3 = augment(class_3, labels_3)
class_4, labels_4 = augment(class_4, labels_4)
class_5, labels_5 = augment(class_5, labels_5)
class_6, labels_6 = augment(class_6, labels_6)
class_7, labels_7 = augment(class_7, labels_7)
class_8, labels_8 = augment_much_more(class_8, labels_8)

In [5]:
X = np.concatenate((class_0, class_1, class_2, class_3, class_4, class_5, class_6, class_7, class_8))
y = np.concatenate((labels_0, labels_1, labels_2, labels_3, labels_4, labels_5, labels_6, labels_7, labels_8))
X.shape, y.shape

((3520, 112, 112, 3), (3520,))

In [8]:
X = normalize(X)

In [9]:
## free tge class variables to save memory

del class_0, class_1, class_2, class_3, class_4, class_5, class_6, class_7, class_8

In [10]:
## splitting the dataset into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [11]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2816, 112, 112, 3), (704, 112, 112, 3), (2816,), (704,))

In [12]:
## VGG 16 embeddings

vgg = VGG16(include_top=False, weights='imagenet', input_shape=(112, 112, 3))
vgg.trainable = False
vgg.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 112, 112, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 112, 112, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 112, 112, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 56, 56, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 28, 28, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 28, 28, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 7, 7, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 7, 7, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 7, 7, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 3, 3, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [13]:
## generating embeddings

X_train = vgg.predict(X_train)
print(X_train.shape)

X_test = vgg.predict(X_test)
print(X_test.shape)

88/88 ━━━━━━━━━━━━━━━━━━━━ 76s 858ms/step
(2816, 3, 3, 512)
22/22 ━━━━━━━━━━━━━━━━━━━━ 21s 976ms/step
(704, 3, 3, 512)


In [14]:
## flatten the embeddings

X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

In [15]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2816, 4608), (704, 4608), (2816,), (704,))

In [18]:
## training the model

rf = RandomForestClassifier(n_estimators=512, random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=512, random_state=42)

In [20]:
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9872159090909091

In [28]:
xgb = XGBClassifier(n_estimators=128, random_state=42, n_jobs = -1)

xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=128, n_jobs=-1,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [29]:
y_pred = xgb.predict(X_test)
accuracy_score(y_test, y_pred)

0.984375

In [32]:
## light gbm

lgb = LGBMClassifier(n_estimators=32, random_state=42, n_jobs = -1)

lgb.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.286984 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 617304
[LightGBM] [Info] Number of data points in the train set: 2816, number of used features: 4129
[LightGBM] [Info] Start training from score -2.024179
[LightGBM] [Info] Start training from score -2.558578
[LightGBM] [Info] Start training from score -2.108262
[LightGBM] [Info] Start training from score -2.171632
[LightGBM] [Info] Start training from score -2.070955
[LightGBM] [Info] Start training from score -2.070955
[LightGBM] [Info] Start training from score -1.971811
[LightGBM] [Info] Start training from score -2.073776
[LightGBM] [Info] Start training from score -3.270244
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

LGBMClassifier(n_estimators=32, n_jobs=-1, random_state=42)

In [33]:
y_pred = lgb.predict(X_test)
accuracy_score(y_test, y_pred)

0.9772727272727273